In [1]:
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 3142

import numpy as np
np.random.seed(algorithm_globals.random_seed)

from qiskit_machine_learning.datasets import ad_hoc_data

train_data, train_labels, test_data, test_labels= (
    ad_hoc_data(training_size=20, test_size=5, n=2, gap=0.3, one_hot=False))

In [2]:
from qiskit.circuit.library import ZZFeatureMap, TwoLocal

adhoc_feature_map = ZZFeatureMap(feature_dimension=2, reps=2)
adhoc_var_form = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)

adhoc_circuit = adhoc_feature_map.compose(adhoc_var_form)
adhoc_circuit.measure_all()
adhoc_circuit.decompose().draw()

┌───┐┌─────────────┐                                          ┌───┐»
   q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■──┤ H ├»
        ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├───┤»
   q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ H ├»
        └───┘└─────────────┘└───┘└──────────────────────────────┘└───┘└───┘»
meas: 2/═══════════════════════════════════════════════════════════════════»
                                                                           »
«        ┌─────────────┐                                          ┌──────────┐»
«   q_0: ┤ P(2.0*x[0]) ├──■────────────────────────────────────■──┤ Ry(θ[0]) ├»
«        ├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├──────────┤»
«   q_1: ┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ Ry(θ[1]) ├»
«        └─────────────┘└───┘└──────────────────────────────┘└───┘└──────────┘»
«meas: 2/═════════════════════════════════════════════════════════════════════»
«                                                                             »
«        ┌──────────┐   ┌──────────┐┌──────────┐   ┌──────────┐┌───────────┐ ░ »
«   q_0: ┤ Rz(θ[2]) ├─■─┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├─■─┤ Ry(θ[8]) ├┤ Rz(θ[10]) ├─░─»
«        ├──────────┤ │ ├──────────┤├──────────┤ │ ├──────────┤├───────────┤ ░ »
«   q_1: ┤ Rz(θ[3]) ├─■─┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├─■─┤ Ry(θ[9]) ├┤ Rz(θ[11]) ├─░─»
«        └──────────┘   └──────────┘└──────────┘   └──────────┘└───────────┘ ░ »
«meas: 2/══════════════════════════════════════════════════════════════════════»
«                                                                              »
«        ┌─┐   
«   q_0: ┤M├───
«        └╥┘┌─┐
«   q_1: ─╫─┤M├
«         ║ └╥┘
«meas: 2/═╩══╩═
«         0  1

In [3]:
def circuit_parameters(data, variational):
    parameters = {}
    for i, p in enumerate(adhoc_feature_map.ordered_parameters):
        parameters[p] = data[i]
    for i, p in enumerate(adhoc_var_form.ordered_parameters):
        parameters[p] = variational[i]
    return parameters

In [4]:
def assign_label(bitstring):
    hamming_weight = sum([int(k) for k in list(bitstring)])
    odd = hamming_weight & 1
    if odd:
        return 0
    else:
        return 1

In [5]:
def label_probability(results):
    shots = sum(results.values())
    probabilities = {0: 0, 1: 0}
    for bitstring, counts in results.items():
        label = assign_label(bitstring)
        probabilities[label] += counts / shots
    return probabilities

In [6]:
from qiskit import BasicAer, execute

def classification_probability(data, variational):
    circuits = [adhoc_circuit.assign_parameters(
        circuit_parameters(d,variational)) for d in data]
    backend = BasicAer.get_backend('qasm_simulator')
    results =  execute(circuits, backend).result()

    classification = [label_probability(results.get_counts(c)) for c in circuits]
    return classification

In [15]:
def cross_entropy_loss(predictions, expected):
    p = predictions.get(expected)
    return -(expected*np.log(p)+(1-expected)*np.log(1-p))
   
def cost_function(data, labels, variational):
    classifications = classification_probability(data, variational)

    cost = 0
    accuracy = 0
    for i, classification in enumerate(classifications):
        cost += cross_entropy_loss(classification, labels[i])
        if (classification[0] >= classification[1]) and (labels[i] == 0):
            accuracy += 1
        elif (classification[1]) >= classification[0] and (labels[i] == 1):
            accuracy += 1
    accuracy /= len(labels)
    cost /= len(data)
    
    print('cost:', np.around(cost, 4), 'acc:', np.around(accuracy,4))

    return cost

In [ ]:
# Callback function for optimiser for plotting purposes
def store_intermediate_result(evaluation, parameter, cost, 
                              stepsize, accept):
    evaluations.append(evaluation)
    parameters.append(parameter)
    costs.append(cost)

# Set up the optimization
from qiskit.algorithms.optimizers import SPSA

parameters = []
costs = []
evaluations = []

optimizer = SPSA(maxiter=100, callback=store_intermediate_result)

#initial_point = np.random.random(adhoc_var_form.num_parameters)
initial_point = np.array([3.28559355, 5.48514978, 5.13099949,
                          0.88372228, 4.08885928, 2.45568528,
                          4.92364593, 5.59032015, 3.66837805,
                          4.84632313, 3.60713748, 2.43546])

objective_function = lambda variational: cost_function(train_data,
                                                       train_labels,
                                                       variational)

# Run the optimization
opt_var, opt_value, _ = optimizer.optimize(len(initial_point), objective_function, initial_point=initial_point)

import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(evaluations, costs)
plt.xlabel('Steps')
plt.ylabel('Cost')
plt.show()

cost: 0.7789 acc: 0.125
cost: 0.8227 acc: 0.6
cost: 1.0044 acc: 0.475
cost: 0.5908 acc: 0.325
cost: 0.7249 acc: 0.45
cost: 0.8071 acc: 0.275
cost: 0.934 acc: 0.4
cost: 0.5951 acc: 0.35
cost: 0.731 acc: 0.2
cost: 0.994 acc: 0.375
cost: 0.6927 acc: 0.5
cost: 0.8532 acc: 0.275
cost: 0.6796 acc: 0.225
cost: 1.0529 acc: 0.425
cost: 0.8354 acc: 0.55
cost: 0.7071 acc: 0.275
cost: 0.8484 acc: 0.525
cost: 0.6275 acc: 0.325
cost: 0.8516 acc: 0.375
cost: 0.6956 acc: 0.3
cost: 0.5914 acc: 0.325
cost: 0.9274 acc: 0.45
cost: 0.9781 acc: 0.45
cost: 0.6595 acc: 0.35
cost: 0.8308 acc: 0.3
cost: 0.7915 acc: 0.45
cost: 0.6642 acc: 0.475
cost: 0.8858 acc: 0.45
cost: 1.0482 acc: 0.5
cost: 0.6817 acc: 0.25
cost: 0.6318 acc: 0.3
cost: 0.9488 acc: 0.425
cost: 0.7675 acc: 0.275
cost: 0.9473 acc: 0.475
cost: 0.6532 acc: 0.325
cost: 1.0137 acc: 0.425
cost: 0.7179 acc: 0.3
cost: 0.8782 acc: 0.5
cost: 0.664 acc: 0.275
cost: 1.0051 acc: 0.45
cost: 0.7499 acc: 0.25
cost: 0.7828 acc: 0.45
cost: 0.8554 acc: 0.35
cost: